## Quiz 04 Answers
1. False
2. True
3. False
4. 3 + 0 + 2 + 7 = 12
5. False

In [ ]:
# Objective 1: Define start and end probability distributions from LSTM outputs for an MRC model

import torch
from torch.nn import Linear, Softmax

torch.manual_seed(605)

batch_size = 2
seq_len = 8
hidden_size = 32

layer = Linear(32, 2)
softmax = Softmax(-1)

lstm_outputs = torch.randn(batch_size, seq_len, hidden_size)

start_logits, end_logits = layer(lstm_outputs).chunk(2, dim=-1)
start_prob = softmax(start_logits.squeeze(-1))
end_prob = softmax(end_logits.squeeze(-1))
assert (start_prob[0, 0] * end_prob[0, 7] - 0.0139).abs() < 0.001 # probability that the answer is the entire sequence

In [ ]:
# Objective 2: Be able to compute L1 distance, L2 distance, inner product, cosine similarity, and cosine distance

import torch

torch.manual_seed(605)

hidden_size = 32
num_a = 4
num_b = 8
a = torch.randn(num_a, hidden_size)
b = torch.randn(num_b, hidden_size)

def cosine_distance(a, b):
  return (a / a.norm(dim=1, keepdim=True)) @ (b.T / b.T.norm(dim=0, keepdim=True))

assert (cosine_distance(a, b)[0, 0] + 0.3874).abs() < 0.001

In [ ]:
# Objective 3: Given a list of documents, be able to define TF-IDF vectors

import torch

torch.manual_seed(605)

num_docs = 32
vocab_size = 64

# Given
documents_tf = (torch.rand(num_docs, vocab_size) * 10).round() # term frequency of documents, in the range of 0 to 10
query_tf = (torch.rand(1, vocab_size) * 3).round()

# Compute tf-idf
df = (documents_tf > 0).float().sum(0)
idf = torch.log(torch.tensor(documents_tf.size(0))) - df.log()
documents_tfidf = documents_tf * idf.unsqueeze(0)
query_tfidf = query_tf * idf.unsqueeze(0)
scores = cosine_distance(query_tfidf, documents_tfidf)
values, indices = scores.max(dim=1)
assert indices[0] == 31